In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import requests
import folium
import json
from pandas.io.json import json_normalize
import folium

In [3]:
# 크롤링 
# 서울따릉이
targetSite = 'https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do'
request =requests.post(targetSite,data = {
    'stationGrpSeq': 'ALL'
})# Request Method: POST이므로 post 메소드 이용
print(type(request.text))
#print(request.text)

<class 'str'>


In [4]:
# json 모듈의 loads() 메소드로 크롤링한 json 타입의 문자열을 파이썬에서 처리하기 위해 딕셔너리 타입으로 변환한다.
bike_json = json.loads(request.text)
print(type(bike_json)) # <class 'dict'>
#print(bike_json)

<class 'dict'>


In [5]:
# json_normalize() 메소드를 사용해서 딕셔너리 타입의 데이터를 판다스 데이터 프레임으로 변환한다.
# json_normalize(딧겨너리, 딕셔너리에서 데이터프레임으로 변경할 데이터와 연결된 키 이름)
bike_df = json_normalize(bike_json, 'realtimeList')
bike_df

,stationName,stationImgFileName,stationId,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,stationSeCd,mode
0,위트콤공장,,ST-598,0.00000000,0.00000000,68,0,0,0,RAK_001,None
1,102. 망원역 1번출구 앞,,ST-4,126.91062927,37.55564880,22,0,15,2,RAK_001,None
2,103. 망원역 2번출구 앞,,ST-5,126.91083527,37.55495071,16,0,5,2,RAK_001,None
3,104. 합정역 1번출구 앞,,ST-6,126.91498566,37.55062866,15,0,0,0,RAK_001,None
4,105. 합정역 5번출구 앞,,ST-7,126.91482544,37.55000687,7,0,1,0,RAK_001,None
...,...,...,...,...,...,...,...,...,...,...,...
2316,4857. 올림픽공원 장미광장,,ST-2621,127.12629700,37.51275635,15,0,2,0,RAK_002,None
2317,4859. 잠실나루 나들목,,ST-2609,127.10028076,37.52148438,15,0,2,1,RAK_002,None
2318,4863. 잠실나들목5,,ST-2728,127.07761383,37.51640320,7,0,0,0,RAK_002,None
2319,상담센터,,ST-1747,0.00000000,0.00000000,4,0,6,0,RAK_001,None


In [6]:
bike_df.columns

Index(['stationName', 'stationImgFileName', 'stationId', 'stationLongitude',
       'stationLatitude', 'rackTotCnt', 'parkingBikeTotCnt',
       'parkingQRBikeCnt', 'parkingELECBikeCnt', 'stationSeCd', 'mode'],
      dtype='object')

In [7]:
# 필요한 column 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터 프레임을 만든다.
# stationName -> 대여소 이름
# stationLatitude -> 대여소 위도
# stationLongitude -> 경도
# rackTotCnt -> 전체 자전거 대수
# parkingBikeTotCnt -> 주차된 따릉이 LCD형 대수
# parkingQRBikeCnt -> 주차된 따릉이 QR형 대수
# parkingELECBikeCnt -> 새싹 따릉이 대수

# 판다스 데이터 프레임에서 특정 컬럼의 데이터를 얻어올 때 2개 이상의 컬럼을 얻어와야 한다면 얻어올 컬럼 목록을 반드시 []로
# 묶어야 한다.
bike_df_map = bike_df[['stationName','stationLatitude','stationLongitude','rackTotCnt','parkingBikeTotCnt','parkingQRBikeCnt'
                       ,'parkingELECBikeCnt']]
bike_df_map

,stationName,stationLatitude,stationLongitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt
0,위트콤공장,0.00000000,0.00000000,68,0,0,0
1,102. 망원역 1번출구 앞,37.55564880,126.91062927,22,0,15,2
2,103. 망원역 2번출구 앞,37.55495071,126.91083527,16,0,5,2
3,104. 합정역 1번출구 앞,37.55062866,126.91498566,15,0,0,0
4,105. 합정역 5번출구 앞,37.55000687,126.91482544,7,0,1,0
...,...,...,...,...,...,...,...
2316,4857. 올림픽공원 장미광장,37.51275635,127.12629700,15,0,2,0
2317,4859. 잠실나루 나들목,37.52148438,127.10028076,15,0,2,1
2318,4863. 잠실나들목5,37.51640320,127.07761383,7,0,0,0
2319,상담센터,0.00000000,0.00000000,4,0,6,0


In [8]:
bike_df_map.dtypes

stationName           object
stationLatitude       object
stationLongitude      object
rackTotCnt            object
parkingBikeTotCnt     object
parkingQRBikeCnt      object
parkingELECBikeCnt    object
dtype: object

In [9]:
bike_df_map['stationLatitude'] = bike_df_map['stationLatitude'].astype(float)
bike_df_map['stationLongitude'] = bike_df_map['stationLongitude'].astype(float)
bike_df_map['parkingBikeTotCnt'] = bike_df_map['parkingBikeTotCnt'].astype(int)
bike_df_map['parkingQRBikeCnt'] = bike_df_map['parkingQRBikeCnt'].astype(int)
bike_df_map['parkingELECBikeCnt'] = bike_df_map['parkingELECBikeCnt'].astype(int)
bike_df_map['total'] = bike_df_map['parkingBikeTotCnt'] + bike_df_map['parkingQRBikeCnt'] + bike_df_map['parkingELECBikeCnt']

In [10]:
bike_df_map.dtypes

stationName            object
stationLatitude       float64
stationLongitude      float64
rackTotCnt             object
parkingBikeTotCnt       int32
parkingQRBikeCnt        int32
parkingELECBikeCnt      int32
total                   int32
dtype: object

In [11]:
bike_df_map

,stationName,stationLatitude,stationLongitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,total
0,위트콤공장,0.000000,0.000000,68,0,0,0,0
1,102. 망원역 1번출구 앞,37.555649,126.910629,22,0,15,2,17
2,103. 망원역 2번출구 앞,37.554951,126.910835,16,0,5,2,7
3,104. 합정역 1번출구 앞,37.550629,126.914986,15,0,0,0,0
4,105. 합정역 5번출구 앞,37.550007,126.914825,7,0,1,0,1
...,...,...,...,...,...,...,...,...
2316,4857. 올림픽공원 장미광장,37.512756,127.126297,15,0,2,0,2
2317,4859. 잠실나루 나들목,37.521484,127.100281,15,0,2,1,3
2318,4863. 잠실나들목5,37.516403,127.077614,7,0,0,0,0
2319,상담센터,0.000000,0.000000,4,0,6,0,6


In [12]:
bike_df_map = bike_df_map.loc[(bike_df_map['stationLatitude']>37) & (bike_df_map['stationLongitude']>125)]
bike_df_map

,stationName,stationLatitude,stationLongitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,total
1,102. 망원역 1번출구 앞,37.555649,126.910629,22,0,15,2,17
2,103. 망원역 2번출구 앞,37.554951,126.910835,16,0,5,2,7
3,104. 합정역 1번출구 앞,37.550629,126.914986,15,0,0,0,0
4,105. 합정역 5번출구 앞,37.550007,126.914825,7,0,1,0,1
5,106. 합정역 7번출구 앞,37.548645,126.912827,12,0,0,0,0
...,...,...,...,...,...,...,...,...
2314,4855. 거여5단지 503동 앞,37.489330,127.146050,10,0,10,0,10
2315,4856. 올림픽공원 서1문 앞,37.520218,127.114571,10,0,4,0,4
2316,4857. 올림픽공원 장미광장,37.512756,127.126297,15,0,2,0,2
2317,4859. 잠실나루 나들목,37.521484,127.100281,15,0,2,1,3


In [13]:
bike_map = folium.Map(location=[bike_df_map['stationLatitude'].mean(),bike_df_map['stationLongitude'].mean()],zoom_start=12)
bike_map

# iterrows() : 데이터프레임의 인덱스와 데이터를 리턴시킨다.
for index,data in bike_df_map.iterrows():
        #print(index,data)
        string = '{} LED : {}대, QR : {}대, 새싹 : {}대'.format(data['stationName'], data['parkingBikeTotCnt'],
                                                          data['parkingQRBikeCnt'], data['parkingELECBikeCnt'])
        string=string[string.index('.')+1:]
        popup = folium.Popup(string,max_width=600)
        folium.Marker(location=[data['stationLatitude'],data['stationLongitude']],
                      popup=popup,
                     icon=folium.Icon(color='green',icon='arrow-down')).add_to(bike_map)

